In [1]:
import sys
import pygame
from pygame.locals import *
import pymunk 

from pymunk.vec2d import Vec2d

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Darwin (64bit) [/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/libchipmunk.dylib]


In [2]:
from pymunk import pygame_util
import random
import math

In [3]:
def add_bullet(space):
    mass = 1
    radius = 5
    moment = pymunk.moment_for_circle(mass, 0, radius) # 1
    body = pymunk.Body(mass, moment) # 2
    body.position = 0, 200 # 3
    body.start_position = Vec2d(body.position)
    shape = pymunk.Circle(body, radius) # 4
    
    shape.elasticity = 0
    shape.friction = 1000000
    shape.collision_type = 1
    
    space.add(body, shape) # 5
    return shape

In [4]:
def add_pendulum(space):
    
    # x and y coords for block's center
    center_x = 300
    center_y = 100
    
    # Add pendulum's block
    #block_mass = 10000    
    #block_moment = pymunk.moment_for_box(block_mass, (200,100))
    block_body = pymunk.Body()
    
    block_body.position = center_x, center_y
    block_shape = pymunk.Poly(block_body, [(center_x-100, center_y-50),(center_x-100, center_y+50),(center_x+100, center_y+50),(center_x+100, center_y-50)])
    block_shape.density = .0025
    
    block_shape.elasticity = 0
    block_shape.friction = 10000000
    block_shape.collision_type = 2
    
    space.add(block_body, block_shape)
    print(block_shape.mass, block_shape.moment)
    
    
    # Make pendulum hang from one string 
    pivot_point = pymunk.Body(body_type = pymunk.Body.STATIC)
    pivot_point.position = (center_x, center_y+100)
    
    joint = pymunk.constraint.PinJoint(pivot_point, block_body, (center_x, center_y+100), (center_x,center_y+50))
    #joint = pymunk.constraint.PinJoint(space.static_body, block_body, (center_x, center_y+100), (0,0))
    space.add(joint)
    
    return block_shape, joint

In [5]:
def bullet_hits(arbiter, space, _):    
    space.remove(space.shapes[0]) # remove the bullet shape
    space.remove(space.bodies[0]) # remove the bullet body
    """ TODO transfer mass and momentum of bullet to the block before removing bullet
    i.e. "bullet goes into block"
    """

In [6]:
def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    pygame.display.set_caption("Ballistic Pendulum Simulation")
    clock = pygame.time.Clock()

    space = pymunk.Space() #2
    space.gravity = 0, -900
    space.damping = .99
    
    bullet = add_bullet(space)
    block, pinjoint = add_pendulum(space)
    
    # Setup bullet-block collision callback function
    h = space.add_collision_handler(1, 2) # ...(COLLTYPE_BULLET, COLLTYPE_BLOCK)
    h.post_solve = bullet_hits
    

    draw_options = pymunk.pygame_util.DrawOptions(screen)

    space.shapes[0].body.apply_impulse_at_local_point((15000,0))
    
    
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                sys.exit(0)
            elif event.type == KEYDOWN and event.key == K_ESCAPE:
                sys.exit(0)
                
                
        space.step(1/500.0) # TEMP: Slow-motion
        
        
        
        screen.fill((255,255,255))
        space.debug_draw(draw_options)


        pygame.display.flip()
        clock.tick(50)

if __name__ == '__main__':
    sys.exit(main())

(50.0, 208333.33333333334)


SystemExit: 0

/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
